# Prepare the LandSat Image 
Convert from float64 to unit16 & nodata values are set to 0

Note to self the landsat directory currently being used is not the correct one. We need the one that was at the planet location..

In [1]:
# Standard Library Imports
import os
import shutil
import glob

# Third-party Library Imports
import numpy as np
import rasterio
from rasterio.plot import show
from skimage.io import imsave
from skimage import img_as_ubyte

# Project-specific Imports
from coastseg_planet import processing
from coastseg_planet import masking
from coastseg_planet import download
from coastseg_planet import model
from coastseg_planet import coregister
from coastseg_planet import visualize

# Specific functions from project-specific modules
from coastseg_planet.processing import (
    convert_from_float_to_unit16,
    create_tiled_raster,
    convert_landsat_to_model_format,
    get_best_landsat_from_dir,
    get_best_landsat_tifs,
    get_tiffs_with_bad_area
)
from coastseg_planet.masking import apply_cloudmask_to_dir


c:\Users\sf230\anaconda3\envs\coastseg_planet\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


Transformers is only compatible with Keras 2, but you have explicitly set `TF_USE_LEGACY_KERAS` to `0`. This may result in unexpected behaviour or errors if Keras 3 objects are passed to Transformers models.


Version:  2.15.0
Eager mode:  True


## Global Variables

In [2]:
# If downloaded with TOAR tool applied don't convert to TOAR again (set this to False)
convert_to_TOAR = False
# Whether to run the classification model or not to sort the images into good and bad directories
run_good_bad_classification = False #@todo make this true

## Initialization
1. Provide directory of CoastSeg downloaded session
2. The directory of planet images

In [3]:
# input the coastseg session with landsat data downloaded
downloaded_directory = r"C:\development\doodleverse\coastseg\CoastSeg\data\ID_1_datetime06-07-24__02_01_53" # this is a custom roi thats the same as the planet roi
# this is where we would download the ROI with planet or load it if it was already downloaded
planet_dir = r"C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene"
good_dir = os.path.join(planet_dir, "good")
model_path = r"C:\development\coastseg-planet\CoastSeg-Planet\models\best_rgb.h5"
csv_path = os.path.join(planet_dir, "classification_results.csv")
output_folder = planet_dir
path_to_inference_imgs = planet_dir

## Get the Best LandSat Raster

In [4]:
# get the landsat with the lowest RMSE and the cloud cover mask
landsat_dir=get_best_landsat_from_dir(downloaded_directory)
landsat_path, raw_landsat_cloud_mask_path=get_best_landsat_tifs(downloaded_directory)
print(f"The best satellite is {landsat_dir}")
print(f"The best landsat tiff is at : {landsat_path}")
print(f"The best landsat cloud mask is at : {raw_landsat_cloud_mask_path}")

54
The best satellite is L9
The best landsat tiff is at : C:\development\doodleverse\coastseg\CoastSeg\data\ID_1_datetime06-07-24__02_01_53\L9\ms\2023-07-23-22-08-13_L9_ID_1_datetime06-07-24__02_01_53_ms.tif
The best landsat cloud mask is at : C:\development\doodleverse\coastseg\CoastSeg\data\ID_1_datetime06-07-24__02_01_53\L9\mask\2023-07-23-22-08-13_L9_ID_1_datetime06-07-24__02_01_53_mask.tif


## Convert the LandSat to RGB Tiff Model Compatible Format

In [5]:
landsat_processed_path  = processing.format_landsat_tiff(landsat_path)
print(f"The landsat image that will be used for the model is at : {landsat_processed_path}")

dtype: uint8, shape: (280, 280, 3), count: 3
Saving image to C:\development\doodleverse\coastseg\CoastSeg\data\ID_1_datetime06-07-24__02_01_53\L9\ms\2023-07-23-22-08-13_L9_ID_1_datetime06-07-24__02_01_53_processed.jpg
The landsat image that will be used for the model is at : C:\development\doodleverse\coastseg\CoastSeg\data\ID_1_datetime06-07-24__02_01_53\L9\ms\2023-07-23-22-08-13_L9_ID_1_datetime06-07-24__02_01_53_processed_model_format.tif


## Convert the LandSat Cloud Mask to a Boolean Format


In [6]:
landsat_cloud_mask_path =  raw_landsat_cloud_mask_path.replace(".tif","_processed.tif")
masking.save_landsat_cloud_mask(raw_landsat_cloud_mask_path, landsat_cloud_mask_path)

'C:\\development\\doodleverse\\coastseg\\CoastSeg\\data\\ID_1_datetime06-07-24__02_01_53\\L9\\mask\\2023-07-23-22-08-13_L9_ID_1_datetime06-07-24__02_01_53_mask_processed.tif'

# Filter the Planet Images By Area
- Remove images that are less than 80% of the expected area & move them to the bad directory

In [7]:
# get the tiffs in the directory depending on whether the TOAR tool was applied or not
tiff_paths= []
if convert_to_TOAR:
    tiff_paths = glob.glob(os.path.join(planet_dir,"*AnalyticMS_clip.tif"))
else: # if the TOAR tool was already applied
    tiff_paths = glob.glob(os.path.join(planet_dir,"*3B_AnalyticMS_toar_clip.tif"))
    if len(tiff_paths) == 0:
        print("No TIFFs found in the directory")
    
print(f"Number of TIFFs: {len(tiff_paths)}")
# improvement get the planet tiff with the largest area and use that as the expected area
expected_area_km = 4.5  # 4.1 square kilometers
threshold = 0.90
bad_tiff_paths = get_tiffs_with_bad_area(tiff_paths, expected_area_km, threshold, use_threads=True)
print(f"Number of bad TIFFs: {len(bad_tiff_paths)}")
# for each bad tiff move it a new folder called bad
bad_path = os.path.join(planet_dir,"bad")
os.makedirs(bad_path,exist_ok=True)
for bad_tiff in bad_tiff_paths:
    # move the file to the bad folder
    shutil.move(bad_tiff,bad_path)

Number of TIFFs: 97
Number of bad TIFFs: 0


# Convert the Planet Images to TOAR if not already TOAR
-todo use the fact the tifs generated by Toar clip in the name to set the value of convert_to_TOAR

In [8]:
# convert the files in the directory to TOAR (Top of Atmosphere Reflectance) 
if convert_to_TOAR:
    processing.convert_directory_to_TOAR(planet_dir,input_suffix='AnalyticMS_clip.tif',output_suffix='_TOAR.tif',separator='_3B')


# Convert Planet Imagery to Same Format as LandSat
- reorder to bands to be  RGBNIR
- scale the values to be 0 - 255 

In [9]:
# convert to RGB format that is compatible with the model. Range of values is 0-255
if convert_to_TOAR:
    input_suffix='3B_TOAR.tif'
else:
    input_suffix='_3B_AnalyticMS_toar_clip.tif'
processing.convert_directory_to_model_format(planet_dir,input_suffix=input_suffix,output_suffix='_TOAR_model_format.tif',separator='_3B')

## Filter the Planet Imagery
Use a model to sort the images as good or bad

In [ ]:
run_good_bad_classification = True

In [10]:

regex = '*TOAR_model_format'
# use a parameter to control if the good bad classification should run again
if run_good_bad_classification:
    model.run_classification_model(model_path, path_to_inference_imgs, output_folder, csv_path, regex, move_files=False)
# if the classification model was not run then 
if not os.path.exists(good_dir):
    good_dir = planet_dir

# Apply Cloud Masks to Planet
Create a cloud and shadow mask for each image in the directory of planet images

In [11]:
# apply cloud mask to directory of planet images and generate cloud masks in that directory for each tif
apply_cloudmask_to_dir(good_dir)

Processing C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_udm2_clip.tif
Creating cloud and shadow mask from C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_udm2_clip.tif
Saving combined mask to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_udm2_clip_combined_mask.tif
Processing C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211939_09_24c3_3B_udm2_clip.tif
Creating cloud and shadow mask from C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211939_09_24c3_3B_udm2_clip.tif
Saving combined mask to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b

# Coregister the Planet Imagery to the LandSat Image
- We will coregister each modified planet image to the landsat image that was modified to be in the zoo format

In [12]:
kwargs = {
   'max_shift':2,
   'align_grids':True,
    "tieP_filter_level": 3,
    "min_reliability": 50,
    "grid_res": 100,  # tie point grid resolution in pixels of the target image (x-direction) (the lower this value the longer it takes)
   'v':True, #verbose mode
   'q':False, #quiet mode
    "r_b4match": 1,  # band of reference image to be used for matching (starts with 1; default: 1)
    "s_b4match": 1,  # band of source image to be used for matching (starts with 1; default: 1)
   'ignore_errors':False, # Recommended to set to True for batch processing
    'progress':True,
    'out_crea_options':["COMPRESS=LZW"], # otherwise if will use deflate which will not work well with our model
    'fmt_out':'GTIFF',
    'CPUs' : coregister.get_cpus()/2,
}

Number of CPUs available: 20


In [13]:
coregister.coregister_directory(good_dir,
                                landsat_processed_path,
                                landsat_cloud_mask_path,
                                output_suffix='_TOAR_processed_coregistered_local.tif',
                                bad_mask_suffix='_combined_mask.tif',
                                use_local=True,  # if True, use local coregistration, otherwise use global coregistration
                                overwrite=False, # if True, will overwrite the existing coregistered files, otherwise they will be skipped
                                **kwargs)

Number of CPUs available: 20
bad_mask_suffix: _combined_mask.tif
input_path: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_TOAR_model_format.tif
Parent directory: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good
output_path: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_TOAR_processed_coregistered_local.tif
Coregistered planet image saved to: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_TOAR_processed_coregistered_local.tif
bad_mask_suffix: _combined_mask.tif
input_path: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211939_09_24c3_3B_TOAR_model_format.tif
Parent directory: C:\deve

# Apply the Model to the Planet Imagery

## Create an Elevation Mask from the Topobathy Data
- We will use this as our reference shoreline buffer
- This code is a bit of a placeholder eventually we will use the code to download the topobathy for the region 
- For now manually provide the topobathy tiff

In [14]:
from coastseg_planet.download import download_topobathy
# get the roi used to download the planet data
config_gdf_path = os.path.join(downloaded_directory,"config_gdf.geojson")
print(f"Reading the ROI from {config_gdf_path}")
roi_gdf = processing.read_roi_from_config(config_gdf_path)
# print(roi_gdf)
sitename = 'Ak_site'
topobathy_tiff = download_topobathy(sitename,roi_gdf,planet_dir)

Reading the ROI from C:\development\doodleverse\coastseg\CoastSeg\data\ID_1_datetime06-07-24__02_01_53\config_gdf.geojson
File already exists: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\Ak_site_topobathy.tif


In [15]:
import rasterio
from ipyleaflet import Map, ImageOverlay
from ipywidgets import  VBox, FloatRangeSlider
import numpy as np
import PIL.Image
from io import BytesIO
import base64

with rasterio.open(topobathy_tiff) as src:
    mask_data = src.read(1)  # Read the first band
    bounds = src.bounds
    transform = src.transform

# Function to create a base64 PNG image from mask data with transparency
def create_masked_image(mask_data, alpha=128):
    # Normalize the mask data to 0-255
    mask_data = (mask_data / mask_data.max() * 255).astype(np.uint8)
    # Create an RGBA image with the red channel filled and alpha channel set by the mask
    rgba_image = np.zeros((mask_data.shape[0], mask_data.shape[1], 4), dtype=np.uint8)
    rgba_image[..., 0] = mask_data  # Red channel
    # Set alpha to the specified value for the masked regions, 0 for the unmasked regions
    rgba_image[..., 3] = np.where(mask_data > 0, alpha, 0)  # Alpha channel (transparency)
    img = PIL.Image.fromarray(rgba_image)
    with BytesIO() as buffer:
        img.save(buffer, format="png")
        return base64.b64encode(buffer.getvalue()).decode('utf-8')

# Create the initial masked image overlay
mask_data = np.flipud(mask_data)  # Flip the mask data vertically
masked_image = create_masked_image(mask_data)
image_overlay = ImageOverlay(
    url="data:image/png;base64," + masked_image,
    bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]]
)

# Create the map
m = Map(center=[(bounds.top + bounds.bottom) / 2, (bounds.left + bounds.right) / 2], zoom=15)
m.add_layer(image_overlay)

# Function to update the mask (simulated here for demonstration purposes)
def update_mask(change):
    global image_overlay
    if not isinstance(change, tuple):
        mask = mask_data < change
        masked_image = create_masked_image(mask)
        image_overlay.url = "data:image/png;base64," + masked_image
    else:
        lower, upper = change
        mask = np.logical_and(mask_data > lower, mask_data < upper)
        masked_image = create_masked_image(mask)
        image_overlay.url = "data:image/png;base64," + masked_image


default_value = (-10, 10)
# Create a bounded slider with two handles
threshold_slider = FloatRangeSlider(
    description='Elevation (m)',
    value=[default_value[0], default_value[1]],
    min=mask_data.min(),
    max=mask_data.max(),
    step=0.1,
)

# Link the slider to the update function
threshold_slider.observe(lambda change: update_mask(change['new']), names='value')

# Display the map and the slider
display(VBox([m, threshold_slider]))

# Initial display
update_mask(default_value)

In [16]:
from coastseg_planet.processing import create_elevation_mask
from coastseg_planet.processing import match_resolution_and_grid,reproject_to_utm

# defaults if you don't want to use the slider for this example
# -2.86,16.032718467712403
low_threshold = threshold_slider.value[0]
high_threshold = threshold_slider.value[1]

masked_tif_path = create_elevation_mask(topobathy_tiff,topobathy_tiff.replace(".tif","_mask.tif"),low_threshold,high_threshold)
print(f"masked_tif_path: {masked_tif_path}")
# convert the mask to the UTM projection (this is the same as the landsat image)
masked_projected_topobathy_tiff = reproject_to_utm(masked_tif_path, masked_tif_path.replace('.tif', '_utm.tif'))
print(f"output_path: {masked_projected_topobathy_tiff }")

masked_tif_path: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\Ak_site_topobathy_mask.tif
output_path: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\Ak_site_topobathy_mask_utm.tif


In [27]:
from coastseg_planet.shoreline_extraction import get_shorelines_from_model
from coastseg_planet.processing import get_date_from_path,TOA_conversion,get_epsg_from_tiff,get_base_filename,convert_planet_to_model_format
from datetime import datetime
from coastseg_planet.model import apply_model_to_image
from coastseg.extracted_shoreline import load_image_labels, load_merged_image_labels, remove_small_objects_and_binarize, get_indices_of_classnames, get_class_mapping
from coastseg import file_utilities
from coastseg_planet import utils
from coastseg_planet import transects
from coastseg_planet import shoreline_extraction

# Future improvement will download & load the model from the coastseg_planet package
# for now hard code the model path & weights directory
model_card_path = r'C:\development\coastseg-planet\CoastSeg-Planet\output_zoo\coastseg_planet\coastseg_planet\config\model_card.yml'
weights_directory = r'C:\development\doodleverse\coastseg\CoastSeg\src\coastseg\downloaded_models\segformer_RGB_4class_8190958'
model_card_path = file_utilities.find_file_by_regex(
    weights_directory, r".*modelcard\.json$"
)
# all this is to create the settings dictionary needed to extract shorelines
out_epsg = get_epsg_from_tiff(landsat_processed_path)
# convert str to int
out_epsg = int(out_epsg)
settings = {
    'output_epsg': out_epsg,
    'min_length_sl': 100,
    'dist_clouds': 50,
    'min_beach_area': 1000,
    'satname': 'planet',
}

separator = '_3B'
count = 0
suffix = f"{separator}_TOAR_processed_coregistered_global"
print(f"{suffix}.tif")
filtered_tiffs = glob.glob(os.path.join(good_dir, f"*{suffix}.tif"))
if len(filtered_tiffs) == 0:
    print("No tiffs found in the directory")
#@todo
# if length is 1 then call a single function to do all this stuff
all_extracted_shorelines = []
# for target_path in filtered_tiffs:
for file_idx, target_path in enumerate(
        glob.glob(os.path.join(good_dir, f"*{suffix}.tif"))
    ):
    count += 1
    if not os.path.exists(target_path):
        print(f"Skipping {target_path} because it does not exist")
        continue
    base_filename = get_base_filename(target_path, separator)
    print(f"Processing {base_filename}")
    # get the processed coregistered file, the cloud mask and the npz file
        
    cloud_mask_path = utils.get_file_path(good_dir, base_filename, "*3B_udm2_clip.tif")
    if not cloud_mask_path:
        print(f"Skipping {target_path} because cloud mask not found")
        continue
    date=get_date_from_path(base_filename)
    print(f"date {date}")
    print(f"Processing {target_path}")
    print(f"Using cloud mask: {cloud_mask_path}")
    if not os.path.exists(target_path):
        print(f"Skipping {target_path} because it does not exist")
        continue
    
    apply_model_to_image(target_path,good_dir,False,False)
    # this is the file generated by the model
    # npz_path = os.path.join(good_dir, 'out', f"{base_filename}_3B_TOAR_processed_coregistered_res.npz")
    npz_path = os.path.join(good_dir, 'out', f"{base_filename}{suffix}_res.npz")
    if not os.path.exists(npz_path):
        print(f"Skipping {npz_path} because it does not exist")
        
    masked_projected_resolution_topobathy_tiff=masked_projected_topobathy_tiff.replace('.tif', f'_matched_resolution_{count}.tif')
    match_resolution_and_grid(masked_projected_topobathy_tiff , target_path,masked_projected_resolution_topobathy_tiff)
    
    save_path = os.path.join(good_dir, 'out')
    # this shoreline is in the UTM projection
    shoreline_gdf = get_shorelines_from_model(cloud_mask_path,target_path,model_card_path,npz_path,date,satname= 'planet', settings = settings,topobathy_path = masked_projected_resolution_topobathy_tiff,
                                          save_path=save_path)
    all_extracted_shorelines.append(shoreline_gdf)

shorelines_gdf = shoreline_extraction.concat_and_sort_geodataframes(all_extracted_shorelines, "date")

# then intersect these shorelines with the transects
shorelines_dict = transects.convert_shoreline_gdf_to_dict(shorelines_gdf)
shorelines_dict = transects.convert_shoreline_gdf_to_dict(shorelines_gdf)

# transform the transects to the output crs and transform the transects to a dict
transects_path = r"C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\transects.geojson"
transects_gdf = gpd.read_file(transects_path)
transects_gdf.to_crs(settings['output_epsg'], inplace=True)
transects_gdf["id"] = transects_gdf.index
transects_gdf["id"] = transects_gdf["id"].astype(str)
transects_dict = transects.get_transect_points_dict(transects_gdf)
# run compute_intersection_QC
transects.compute_intersection_QC(shorelines_dict, transects_dict)

# from here you can save all the data to csvs


32604
_3B_TOAR_processed_coregistered_global.tif
Processing 20230927_211936_77_24c3
date 2023-09-27-21-19-36
Processing C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_TOAR_processed_coregistered_global.tif
Using cloud mask: C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_udm2_clip.tif


Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing image:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_TOAR_processed_coregistered_global.tif
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211936_77_24c3_3B_TOAR_processed_coregistered_global_predseg.png
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211936_77_24c3_3B_TOAR_processed_coregistered_global_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211936_77_24c3_3B_TOAR_processed_coregistered_global_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829

Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing image:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211939_09_24c3_3B_TOAR_processed_coregistered_global.tif
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20230927_211939_09_24c3_3B_TOAR_processed_coregistered_global_predseg.png
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211939_09_24c3_3B_TOAR_processed_coregistered_global_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20230927_211939_09_24c3_3B_TOAR_processed_coregistered_global_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829

...

Processing image:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20231029_212230_99_2427_3B_TOAR_processed_coregistered_global.tif
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20231029_212230_99_2427_3B_TOAR_processed_coregistered_global_predseg.png
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231029_212230_99_2427_3B_TOAR_processed_coregistered_global_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231029_212230_99_2427_3B_TOAR_processed_coregistered_global_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829

Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing image:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20231102_212052_38_24ce_3B_TOAR_processed_coregistered_global.tif
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\20231102_212052_38_24ce_3B_TOAR_processed_coregistered_global_predseg.png
Output file:  C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231102_212052_38_24ce_3B_TOAR_processed_coregistered_global_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False
N_DATA_BANDS: 3
saving segmentation results to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829db3f\PSScene\good\out\20231102_212052_38_24ce_3B_TOAR_processed_coregistered_global_predseg.png
writing metadata to C:\development\coastseg-planet\downloads\Alaska_TOAR_enabled\42444c87-d914-4330-ba4b-3c948829

Computing transect shoreline intersections: 100%|██████████| 9/9 [00:00<00:00, 271.19it/s]


{'0': array([         nan,          nan, 108.43826017,          nan,
                 nan,          nan,          nan,          nan,
                 nan, 162.63683688,          nan, 130.63472921,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan, 261.50607073,
                 nan, 143.23898858,          nan,          nan,
                 nan,          nan, 136.44552414,          nan,
                 nan,          nan,  92.77896655,          nan,
                 nan,          nan, 103.28968011,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan,
        200.1294093 ,          nan,          nan,          nan,
        145.99487373,          nan,          nan,          nan,
                 nan,          nan, 173.63894745,          nan,
                 nan,          nan,